In [1]:
import pandas as pd
import numpy as np
import sys
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col, mean, min, max
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, GBTRegressor, RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

%load_ext autotime

In [2]:
#sc = SQLContext()
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
       .map(lambda line: line.split(','))
   #didn’t work with take(1). believe returns
   #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

pandas_df = pd.read_csv('f_clusters6.csv')
df = sqlContext.createDataFrame(pandas_df)

#here’s the new vectorizer function:

def vectorizeData(data):
        return data.rdd.map(lambda r: [r[0], r[1], r[2], r[3], Vectors.dense(r[4:-1]),float(r[-1])])\
            .toDF(['route cluster', 'performance cluster', 'userid', 'workoutid', 'features', 'label'])

time: 3.71 s


In [3]:
df.describe('elapsed_time').show()

+-------+-----------------+
|summary|     elapsed_time|
+-------+-----------------+
|  count|            24642|
|   mean|4371.132132132132|
| stddev|9182.794051238823|
|    min|              501|
|    max|          1108301|
+-------+-----------------+

time: 2.12 s


In [4]:
df.schema

StructType(List(StructField(Unnamed: 0,LongType,true),StructField(route_prediction,LongType,true),StructField(perf_prediction,LongType,true),StructField(diff_altitude,DoubleType,true),StructField(geo_distance,DoubleType,true),StructField(userid,LongType,true),StructField(workoutid,LongType,true),StructField(heart_rate_avg,DoubleType,true),StructField(speed_avg,DoubleType,true),StructField(elapsed_time,LongType,true)))

time: 3.32 ms


In [5]:
select_columns = ['route_prediction', 'perf_prediction','userid', 'workoutid', 'geo_distance', 'diff_altitude', \
                'speed_avg', 'heart_rate_avg', 'elapsed_time']
df = df.select(select_columns)

time: 40.5 ms


In [12]:
df.schema

StructType(List(StructField(route_prediction,LongType,true),StructField(perf_prediction,LongType,true),StructField(userid,LongType,true),StructField(workoutid,LongType,true),StructField(geo_distance,DoubleType,true),StructField(diff_altitude,DoubleType,true),StructField(speed_avg,DoubleType,true),StructField(heart_rate_avg,DoubleType,true),StructField(elapsed_time,LongType,true)))

time: 2.37 ms


In [7]:
reg_df = vectorizeData(df)

time: 238 ms


In [13]:
reg_df.schema

StructType(List(StructField(route cluster,LongType,true),StructField(performance cluster,LongType,true),StructField(userid,LongType,true),StructField(workoutid,LongType,true),StructField(features,VectorUDT,true),StructField(label,DoubleType,true)))

time: 2.33 ms


In [14]:
#distinct_clusters = reg_df.select('cluster').distinct().collect()
#cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]

route_clusters = reg_df.select('route cluster').distinct().collect()
perf_clusters = reg_df.select('performance cluster').distinct().collect()
route_cluster_numbers = [int(route_clusters[i][0]) for i in range(len(route_clusters))]
perf_cluster_numbers = [int(perf_clusters[i][0]) for i in range(len(perf_clusters))]

time: 663 ms


In [15]:
reg_df.count()

24642

time: 215 ms


In [ ]:
#(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [28]:
#baseline with defaults
model_list = [LinearRegression(featuresCol="features", labelCol="label"),\
              DecisionTreeRegressor(featuresCol="features", labelCol="label"), \
              GBTRegressor(featuresCol="features", labelCol="label"),\
              RandomForestRegressor(featuresCol="features", labelCol="label")]



for k in model_list:    
    for n in perf_cluster_numbers:
        for i in route_cluster_numbers:
            #print("(Route, Perf, Model): " + str((n,i,k)))
            temp_df1 = reg_df[reg_df['route cluster']==i]
            temp_df = temp_df1[reg_df['performance cluster']==n]
            
            if temp_df.count() == 0:
                #print "Cluster pair dropped"
                pass
            
            else:
                temp_lr = k
                temp_lrModel = temp_lr.fit(temp_df['label','features'])
                temp_df = temp_lrModel.transform(temp_df)

                #paramGrid = ParamGridBuilder() \
                #.addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
                #.addGrid(lr.regParam, [0.1, 0.01]) \
                #.build()



                evaluator = RegressionEvaluator(
                    labelCol="label", predictionCol="prediction", metricName="rmse")

                #crossval = CrossValidator(estimator=pipeline,
                #                          estimatorParamMaps=paramGrid,
                #                          evaluator=BinaryClassificationEvaluator(),
                #                          numFolds=3)  # use 3+ folds in practice#

                # Run cross-validation, and choose the best set of parameters.
                #cvModel = crossval.fit(training)

                #pred = cvModel.transform(temp_df)

                #Print the coefficients and intercept for linear regression
                #print("Coefficients: " + str(temp_lrModel.coefficients))
                #print("Intercept: " + str(temp_lrModel.intercept))



                rmse = evaluator.evaluate(temp_df)
                print("(Route, Perf, Model): " + str((n,i,k)) +": " + str(rmse))
    print "\n\n"

(Route, Perf, Model): (0, 0, LinearRegression_4569ad2e01639820201e): 5708.4075864
(Route, Perf, Model): (0, 1, LinearRegression_4569ad2e01639820201e): 866.367218946
(Route, Perf, Model): (0, 3, LinearRegression_4569ad2e01639820201e): 1000.64071002
(Route, Perf, Model): (0, 2, LinearRegression_4569ad2e01639820201e): 2333.12314457
(Route, Perf, Model): (1, 0, LinearRegression_4569ad2e01639820201e): 9271.03566894
(Route, Perf, Model): (1, 1, LinearRegression_4569ad2e01639820201e): 2989.91882036
(Route, Perf, Model): (1, 3, LinearRegression_4569ad2e01639820201e): 1659.23441376
(Route, Perf, Model): (1, 2, LinearRegression_4569ad2e01639820201e): 2392.20886312
(Route, Perf, Model): (1, 4, LinearRegression_4569ad2e01639820201e): 1955.90771191
(Route, Perf, Model): (2, 0, LinearRegression_4569ad2e01639820201e): 12476.201842
(Route, Perf, Model): (2, 1, LinearRegression_4569ad2e01639820201e): 1286.44133045
(Route, Perf, Model): (2, 3, LinearRegression_4569ad2e01639820201e): 874.70926226
(Route,

In [29]:
#LinearRegression
#masa edit w/ CV
model_list = [LinearRegression(featuresCol="features", labelCol="label")]#,\
              #DecisionTreeRegressor(featuresCol="features", labelCol="label"), \
              #GBTRegressor(featuresCol="features", labelCol="label"),\
              #RandomForestRegressor(featuresCol="features", labelCol="label")]

k = model_list[0]

for n in perf_cluster_numbers:
    for i in route_cluster_numbers:
        #print("(Route, Perf, Model): " + str((n,i,k)))
        temp_df1 = reg_df[reg_df['route cluster']==i]
        temp_df = temp_df1[reg_df['performance cluster']==n]
        temp_df_cv = temp_df

        if temp_df.count() == 0:
            #print "Cluster pair dropped"
            pass
            
        else:
            temp_lr = k
            temp_lrModel = temp_lr.fit(temp_df['label','features'])
            temp_df = temp_lrModel.transform(temp_df)

            paramGrid = ParamGridBuilder() \
            .addGrid(temp_lr.maxIter, [5, 10, 100]) \
            .addGrid(temp_lr.regParam, [0, 0.1, 0.01]) \
            .build()


            evaluator = RegressionEvaluator(
                labelCol="label", predictionCol="prediction", metricName="rmse")

            crossval = CrossValidator(estimator=temp_lr,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=evaluator,
                                      numFolds=10)  # use 3+ folds in practice

            # Run cross-validation, and choose the best set of parameters.
            cvModel = crossval.fit(temp_df_cv)

            pred = cvModel.transform(temp_df_cv)

            print("For " + str(type(temp_lr)) + " and cluster " + str(i))
            #Print the coefficients and intercept for linear regression
            #print("Coefficients: " + str(temp_lrModel.coefficients))
            #print("Intercept: " + str(temp_lrModel.intercept))



            rmse = evaluator.evaluate(pred)
            print("(Route, Perf, Model): " + str((n,i,k)) +": " + str(rmse))


For <class 'pyspark.ml.regression.LinearRegression'> and cluster 0
(Route, Perf, Model): (0, 0, LinearRegression_41c082fb7598100478d7): 5708.40758648
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 1
(Route, Perf, Model): (0, 1, LinearRegression_41c082fb7598100478d7): 866.367226622
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 3
(Route, Perf, Model): (0, 3, LinearRegression_41c082fb7598100478d7): 1000.64071002
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 2
(Route, Perf, Model): (0, 2, LinearRegression_41c082fb7598100478d7): 2333.12314615
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 0
(Route, Perf, Model): (1, 0, LinearRegression_41c082fb7598100478d7): 9271.03566899
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 1
(Route, Perf, Model): (1, 1, LinearRegression_41c082fb7598100478d7): 2989.9188208
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 3
(Route, Perf, Model): (1, 3, Linea

In [30]:
#DecisionTreeRegressor
#masa edit w/ CV
#[LinearRegression(featuresCol="features", labelCol="label"),\
model_list = [DecisionTreeRegressor(featuresCol="features", labelCol="label",maxMemoryInMB=2056)]#, \
              #GBTRegressor(featuresCol="features", labelCol="label"),\
              #RandomForestRegressor(featuresCol="features", labelCol="label")]


k = model_list[0]

for n in perf_cluster_numbers:
    for i in route_cluster_numbers:
        #print("(Route, Perf, Model): " + str((n,i,k)))
        temp_df1 = reg_df[reg_df['route cluster']==i]
        temp_df = temp_df1[reg_df['performance cluster']==n]
        temp_df_cv = temp_df

        if temp_df.count() == 0:
            #print "Cluster pair dropped"
            pass
            
        else:
            temp_lr = k
            temp_lrModel = temp_lr.fit(temp_df['label','features'])
            temp_df = temp_lrModel.transform(temp_df)
        
            paramGrid = ParamGridBuilder() \
            .addGrid(temp_lr.maxDepth, [3, 5]) \
            .addGrid(temp_lr.minInfoGain, [0, 0.1, 1]) \
            .build()


            evaluator = RegressionEvaluator(
                labelCol="label", predictionCol="prediction", metricName="rmse")

            crossval = CrossValidator(estimator=temp_lr,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=evaluator,
                                      numFolds=10)  # use 3+ folds in practice

            # Run cross-validation, and choose the best set of parameters.
            cvModel = crossval.fit(temp_df_cv)

            pred = cvModel.transform(temp_df_cv)


            #print("For " + str(type(temp_lr)) + " and cluster " + str(i))
            #print(cvModel.explainParams())

            #Print the coefficients and intercept for linear regression
            #print("Coefficients: " + str(temp_lrModel.coefficients))
            #print("Intercept: " + str(temp_lrModel.intercept))



            rmse = evaluator.evaluate(pred)
            print("(Route, Perf, Model): " + str((n,i,k)) +": " + str(rmse))


For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 0
(Route, Perf, Model): (0, 0, DecisionTreeRegressor_4f0497d7220292fec819): 5582.01486875
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 1
(Route, Perf, Model): (0, 1, DecisionTreeRegressor_4f0497d7220292fec819): 627.553511351
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 3
(Route, Perf, Model): (0, 3, DecisionTreeRegressor_4f0497d7220292fec819): 945.287293639
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 2
(Route, Perf, Model): (0, 2, DecisionTreeRegressor_4f0497d7220292fec819): 2289.71520794
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 0
(Route, Perf, Model): (1, 0, DecisionTreeRegressor_4f0497d7220292fec819): 8994.42132061
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 1
(Route, Perf, Model): (1, 1, DecisionTreeRegressor_4f0497d7220292fec819): 2979.04526835
For <class 'pyspark.ml.regression.Decisi

In [20]:
print("(Route, Perf, Model): (%d,%d,%s) dropped" % (n,i,k))


(Route, Perf, Model): (0,0,GBTRegressor_47c8b1ce325ef2a47503) dropped
time: 1.08 ms


In [18]:
#Gradient Boosted Trees
#masa edit w/ CV
#[LinearRegression(featuresCol="features", labelCol="label"),\
#DecisionTreeRegressor(featuresCol="features", labelCol="label",maxMemoryInMB=1028)]#, \             
model_list = [GBTRegressor(featuresCol="features", labelCol="label", maxMemoryInMB=4128,lossType="absolute")]#,\
              #RandomForestRegressor(featuresCol="features", labelCol="label")]
k = model_list[0]

f = open('GBT_Larger_Cluster_Regression_Output.txt', 'w')
    





for n in perf_cluster_numbers:
    for i in route_cluster_numbers:
        #print("(Route, Perf, Model): " + str((n,i,k)))
        temp_df1 = reg_df[reg_df['route cluster']==i]
        temp_df = temp_df1[reg_df['performance cluster']==n]
        temp_df_cv = temp_df

        if temp_df.count() == 0:
            #print "Cluster pair dropped"
            print("(Route, Perf, Model): (%d,%d,%s) dropped" % (n,i,k))
            f.write("(Route, Perf, Model): (%d,%d,%s) dropped" % (n,i,k))
            pass
            
        else:
            temp_lr = k
            temp_lrModel = temp_lr.fit(temp_df['label','features'])
            temp_df = temp_lrModel.transform(temp_df)
        
            paramGrid = ParamGridBuilder() \
            .addGrid(temp_lr.maxDepth, [3, 5]) \
            .addGrid(temp_lr.maxIter, [10,20,40]) \
            .build()


            evaluator = RegressionEvaluator(
                labelCol="label", predictionCol="prediction", metricName="rmse")

            crossval = CrossValidator(estimator=temp_lr,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=evaluator,
                                      numFolds=10)  # use 3+ folds in practice

            # Run cross-validation, and choose the best set of parameters.
            cvModel = crossval.fit(temp_df_cv)

            pred = cvModel.transform(temp_df_cv)


            #print("For " + str(type(temp_lr)) + " and cluster " + str(i))
            #print(cvModel.explainParams())

            #Print the coefficients and intercept for linear regression
            #print("Coefficients: " + str(temp_lrModel.coefficients))
            #print("Intercept: " + str(temp_lrModel.intercept))



            rmse = evaluator.evaluate(pred)
            print("(Route, Perf, Model): " + str((n,i,k)) +": " + str(rmse))
            f.write("(Route, Perf, Model): " + str((n,i,k)) +": " + str(rmse) +"\n")
            
f.close()

KeyboardInterrupt: 

time: 3.35 s


In [ ]:
results = {}


In [21]:
#Random Forest Regressor
#masa edit w/ CV
#[LinearRegression(featuresCol="features", labelCol="label"),\
#DecisionTreeRegressor(featuresCol="features", labelCol="label",maxMemoryInMB=1028)], \             
#GBTRegressor(featuresCol="features", labelCol="label", maxMemoryInMB=2056)],\
model_list = [RandomForestRegressor(featuresCol="features", labelCol="label", maxMemoryInMB = 4122)]


k = model_list[0]

for n in perf_cluster_numbers:
    for i in route_cluster_numbers:
        temp_df = reg_df[(reg_df['route cluster'] == i) & (reg_df['performance cluster'] == n)]
        #temp_df1 = reg_df[reg_df['route cluster']==i]
        #temp_df = temp_df1[reg_df['performance cluster']==n]
        temp_df_cv = temp_df

        if temp_df.count() == 0:
            print("(Route, Perf, Model): (%d,%d,%s) dropped" % (n,i,k))
            f.write("(Route, Perf, Model): (%d,%d,%s) dropped\n" % (n,i,k))
            pass
            
        else:
            temp_lr = k
            temp_lrModel = temp_lr.fit(temp_df['label','features'])
            temp_df = temp_lrModel.transform(temp_df)
        
            paramGrid = ParamGridBuilder() \
            .addGrid(temp_lr.maxDepth, [3, 5]) \
            .addGrid(temp_lr.numTrees, [10,20,40]) \
            .build()


            evaluator = RegressionEvaluator(
                labelCol="label", predictionCol="prediction", metricName="rmse")

            crossval = CrossValidator(estimator=temp_lr,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=evaluator,
                                      numFolds=10)  # use 3+ folds in practice

            # Run cross-validation, and choose the best set of parameters.
            cvModel = crossval.fit(temp_df_cv)

            pred = cvModel.transform(temp_df_cv)


            #print("For " + str(type(temp_lr)) + " and cluster " + str(i))
            #print(cvModel.explainParams())

            #Print the coefficients and intercept for linear regression
            #print("Coefficients: " + str(temp_lrModel.coefficients))
            #print("Intercept: " + str(temp_lrModel.intercept))



            rmse = evaluator.evaluate(pred)
            print("(Route, Perf, Model): " + str((n,i,k)) +": " + str(rmse))
            f.write("(Route, Perf, Model): " + str((n,i,k)) +": " + str(rmse) +"\n")
            
f.close()

(Route, Perf, Model): (0, 0, RandomForestRegressor_446490fbdefa6279d5c7): 5568.40799134
(Route, Perf, Model): (0, 1, RandomForestRegressor_446490fbdefa6279d5c7): 681.056741435
(Route, Perf, Model): (0, 3, RandomForestRegressor_446490fbdefa6279d5c7): 658.119447519
(Route, Perf, Model): (0, 2, RandomForestRegressor_446490fbdefa6279d5c7): 1751.27584133


ValueError: unsupported format character 'w' (0x77) at index 30

time: 5min 52s
